# 03 内容分析：情感、叙事与政治立场

**研究核心**: Charlie Kirk政治暗杀事件后72小时的社交媒体舆论内容分析

**分析维度**:
1. **6维情感分析**: sadness, anger, fear, surprise, disgust, joy
2. **6大叙事框架**: 政治暴力受害者、言论后果、政治极化、言论自由、阴谋论、纪念遗产
3. **政治立场分类**: conservative, liberal, neutral
4. **时间演变**: 情感与叙事随72小时的变化
5. **代表性内容**: 每类叙事的典型推文

In [9]:
import sys
from pathlib import Path

# 将项目根目录添加到 Python 路径
project_root = Path('/workspace')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
print(f"✅ Python 路径已配置: {project_root}")

✅ Python 路径已配置: /workspace


## 步骤 1: 加载数据并采样

In [10]:
import polars as pl
import numpy as np
from pathlib import Path

# 加载enriched数据（包含event_time_delta_hours和time_window字段）
df = pl.read_parquet("../parquet/tweets_enriched.parquet")
print(f"📊 数据加载完成: {df.height:,} 行")

# 过滤有效英文文本
df_text = df.filter(
    (pl.col('text').is_not_null()) & 
    (pl.col('lang') == 'en') &
    (pl.col('text').str.len_chars() > 20)  # 至少20字符
)
print(f"📝 有效英文推文: {df_text.height:,} 条")

# 采样策略：每个时间窗口采样最多2000条（确保时间演变分析的代表性）
sample_per_window = 2000

# 方法：对每个时间窗口分别采样后合并
sampled_dfs = []
for window in df_text['time_window'].unique().sort():
    window_df = df_text.filter(pl.col('time_window') == window)
    # 如果该窗口数据少于2000，全部使用；否则采样2000
    n_sample = min(sample_per_window, window_df.height)
    sampled = window_df.sample(n=n_sample, seed=42)
    sampled_dfs.append(sampled)
    print(f"  {window}: {window_df.height:,} 条 → 采样 {n_sample:,} 条")

df_sample = pl.concat(sampled_dfs).sort('createdAt')

print(f"\n📋 采样完成: {df_sample.height:,} 条推文")
print(f"\n时间窗口分布:")
print(df_sample.group_by('time_window').agg(pl.len().alias('count')).sort('time_window'))

📊 数据加载完成: 508,954 行
📝 有效英文推文: 415,714 条
  48-72h: 415,714 条 → 采样 2,000 条

📋 采样完成: 2,000 条推文

时间窗口分布:
shape: (1, 2)
┌─────────────┬───────┐
│ time_window ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
│ 48-72h      ┆ 2000  │
└─────────────┴───────┘


## 步骤 2: 六维情感分析

使用 HuggingFace `j-hartmann/emotion-english-distilroberta-base` 模型  
分类：sadness, joy, love, anger, fear, surprise

In [11]:
from transformers import pipeline
import torch

print("🤖 加载情感分析模型...")
device = 0 if torch.cuda.is_available() else -1
emotion_classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    device=device,
    top_k=None  # 返回所有情感的概率
)

print(f"✅ 模型加载完成 (device: {'GPU' if device == 0 else 'CPU'})")

# 处理文本（批量推理）
texts = df_sample['text'].to_list()
print(f"\n🔄 开始情感分析 ({len(texts):,} 条推文)...")

# 批量处理，每批128条
batch_size = 128
all_emotions = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    # 截断长文本
    batch_truncated = [t[:512] for t in batch]
    results = emotion_classifier(batch_truncated)
    all_emotions.extend(results)
    
    if (i + batch_size) % 1000 == 0:
        print(f"  处理进度: {i + batch_size:,} / {len(texts):,}")

print(f"✅ 情感分析完成")

# 提取主要情感和置信度
primary_emotions = [max(e, key=lambda x: x['score'])['label'] for e in all_emotions]
primary_scores = [max(e, key=lambda x: x['score'])['score'] for e in all_emotions]

# 提取6大情感的分数（构建情感向量）
emotion_labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
emotion_vectors = {}
for label in emotion_labels:
    scores = []
    for result in all_emotions:
        score_dict = {item['label']: item['score'] for item in result}
        scores.append(score_dict.get(label, 0.0))
    emotion_vectors[f'emotion_{label}'] = scores

# 添加到dataframe
df_sample = df_sample.with_columns([
    pl.Series('primary_emotion', primary_emotions),
    pl.Series('emotion_confidence', primary_scores),
    *[pl.Series(k, v) for k, v in emotion_vectors.items()]
])

print(f"\n📊 情感分布:")
print(df_sample.group_by('primary_emotion').agg(pl.len().alias('count')).sort('count', descending=True))

🤖 加载情感分析模型...


Device set to use cpu


✅ 模型加载完成 (device: CPU)

🔄 开始情感分析 (2,000 条推文)...
✅ 情感分析完成

📊 情感分布:
shape: (7, 2)
┌─────────────────┬───────┐
│ primary_emotion ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ anger           ┆ 463   │
│ neutral         ┆ 449   │
│ fear            ┆ 394   │
│ sadness         ┆ 268   │
│ surprise        ┆ 207   │
│ joy             ┆ 141   │
│ disgust         ┆ 78    │
└─────────────────┴───────┘


## 步骤 3: 六大叙事框架检测

基于关键词和语义相似度的叙事分类：
1. **political_violence**: 政治暴力受害者叙事
2. **consequences**: 言论后果叙事
3. **polarization**: 政治极化叙事
4. **free_speech**: 言论自由叙事
5. **conspiracy**: 阴谋论叙事
6. **memorial**: 纪念与遗产叙事

In [12]:
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

print("🤖 加载语义模型用于叙事检测...")
# 复用之前的模型或加载轻量级模型
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

# 定义6大叙事框架的原型文本（核心语义描述）
narrative_prototypes = {
    'political_violence': [
        "This is a tragic political assassination and act of violence",
        "Charlie Kirk was a victim of political violence and murder",
        "The shooting was a terrible attack on a political figure",
        "This assassination is an act of terror against conservatives"
    ],
    'consequences': [
        "His hateful rhetoric had dangerous consequences",
        "This is the result of divisive and toxic speech",
        "He deserves blame for spreading hate and division",
        "His inflammatory words caused this violence"
    ],
    'polarization': [
        "America is deeply divided and polarized",
        "This shows our country is on the brink of civil war",
        "We treat each other as enemies instead of fellow citizens",
        "Political tribalism is tearing our nation apart"
    ],
    'free_speech': [
        "This is an attack on free speech and open debate",
        "They are trying to silence conservative voices",
        "We must defend the right to express political views",
        "Censorship and suppression of speech led to this"
    ],
    'conspiracy': [
        "This was a false flag operation and setup",
        "The deep state planned this assassination",
        "This is a psyop to manipulate public opinion",
        "The official story is fake and a coverup"
    ],
    'memorial': [
        "We honor and remember Charlie Kirk's legacy",
        "His impact on conservative youth will not be forgotten",
        "Rest in peace, he made a difference in politics",
        "We pay tribute to his memory and contributions"
    ]
}

# 生成叙事原型的embeddings（每个叙事用其原型文本的平均embedding）
print("🔢 生成叙事框架语义向量...")
narrative_embeddings = {}
for narrative, prototype_texts in narrative_prototypes.items():
    proto_embs = semantic_model.encode(prototype_texts)
    # 使用平均向量作为该叙事的代表
    narrative_embeddings[narrative] = np.mean(proto_embs, axis=0)

# 关键词辅助（用于增强confidence）
narrative_keywords = {
    'political_violence': [
        r'\bvictim\b', r'\btragedy\b', r'\bassassinat\w*\b', r'\bviolence\b', 
        r'\bmurder\w*\b', r'\bkill\w*\b', r'\bshot\b', r'\bshooting\b',
        r'\bterror\w*\b', r'\bgunman\b', r'\battack\w*\b'
    ],
    'consequences': [
        r'\brhetoric\b', r'\bconsequences\b', r'\bhate speech\b', r'\bdivisive\b',
        r'\bresponsib\w*\b', r'\bblame\b', r'\bcaused\b', r'\bdeserve\w*\b',
        r'\bkarma\b', r'\breap\w*\b'
    ],
    'polarization': [
        r'\bdivided\b', r'\bpolari\w*\b', r'\bcivil war\b', r'\benemy\b',
        r'\bus vs them\b', r'\btear\w* apart\b', r'\bpartisan\b'
    ],
    'free_speech': [
        r'\bfree speech\b', r'\bsilenc\w*\b', r'\bcensor\w*\b', r'\bdebate\b',
        r'\bfirst amendment\b', r'\bvoice\b', r'\bspeak\w* out\b'
    ],
    'conspiracy': [
        r'\bfalse flag\b', r'\bsetup\b', r'\bdeep state\b', r'\bpsyop\b',
        r'\bcoverup\b', r'\bcover-up\b', r'\bplanned\b', r'\binside job\b',
        r'\bfake\b', r'\bhoax\b'
    ],
    'memorial': [
        r'\blegacy\b', r'\bremember\b', r'\bhonor\b', r'\bimpact\b',
        r'\bRIP\b', r'\brest in peace\b', r'\bmemory\b', r'\bmemorial\b',
        r'\btribute\b', r'\bmiss\w*\b'
    ]
}

def detect_narratives_semantic(text, text_embedding):
    """基于语义相似度 + 关键词增强的叙事检测"""
    text_lower = text.lower()
    narrative_scores = {}
    
    for narrative in narrative_prototypes.keys():
        # 1. 语义相似度分数（主要）
        similarity = cosine_similarity(
            text_embedding.reshape(1, -1),
            narrative_embeddings[narrative].reshape(1, -1)
        )[0][0]
        
        # 2. 关键词匹配分数（辅助增强）
        keyword_matches = sum(1 for pattern in narrative_keywords[narrative] 
                             if re.search(pattern, text_lower))
        keyword_boost = keyword_matches * 0.05  # 每个关键词增加5%
        
        # 综合分数：语义相似度为主，关键词提供boost
        final_score = similarity + keyword_boost
        narrative_scores[narrative] = final_score
    
    return narrative_scores

print("🔍 开始基于语义的叙事框架检测...")
print("  (使用sentence embeddings + 关键词增强)")

# 生成所有推文的embeddings（批量处理）
print(f"\n🔢 生成推文语义向量 ({len(texts):,} 条)...")
tweet_embeddings = semantic_model.encode(texts, show_progress_bar=True, batch_size=128)

# 对每条推文进行叙事检测
print("\n🎯 检测叙事框架...")
narrative_results = []
for i, (text, embedding) in enumerate(zip(texts, tweet_embeddings)):
    scores = detect_narratives_semantic(text, embedding)
    narrative_results.append(scores)
    
    if (i + 1) % 2000 == 0:
        print(f"  处理进度: {i + 1:,} / {len(texts):,}")

# 提取主导叙事（得分最高的，且高于阈值0.3）
primary_narratives = []
narrative_confidences = []
for scores in narrative_results:
    max_narrative = max(scores, key=scores.get)
    max_score = scores[max_narrative]
    
    if max_score > 0.3:  # 置信度阈值
        primary_narratives.append(max_narrative)
        narrative_confidences.append(max_score)
    else:
        primary_narratives.append('none')  # 无明显叙事
        narrative_confidences.append(0.0)

# 添加叙事分数列
narrative_cols = {}
for narrative in narrative_prototypes.keys():
    narrative_cols[f'narrative_{narrative}'] = [r[narrative] for r in narrative_results]

df_sample = df_sample.with_columns([
    pl.Series('primary_narrative', primary_narratives),
    pl.Series('narrative_confidence', narrative_confidences),
    *[pl.Series(k, v) for k, v in narrative_cols.items()]
])

print(f"\n✅ 叙事框架检测完成")
print(f"\n📊 叙事分布:")
print(df_sample.group_by('primary_narrative').agg(pl.len().alias('count')).sort('count', descending=True))

print(f"\n📈 平均置信度: {np.mean([c for c in narrative_confidences if c > 0]):.3f}")

🤖 加载语义模型用于叙事检测...
🔢 生成叙事框架语义向量...
🔍 开始基于语义的叙事框架检测...
  (使用sentence embeddings + 关键词增强)

🔢 生成推文语义向量 (2,000 条)...


Batches: 100%|██████████| 16/16 [00:07<00:00,  2.06it/s]



🎯 检测叙事框架...
  处理进度: 2,000 / 2,000

✅ 叙事框架检测完成

📊 叙事分布:
shape: (7, 2)
┌────────────────────┬───────┐
│ primary_narrative  ┆ count │
│ ---                ┆ ---   │
│ str                ┆ u32   │
╞════════════════════╪═══════╡
│ political_violence ┆ 950   │
│ memorial           ┆ 660   │
│ none               ┆ 236   │
│ free_speech        ┆ 68    │
│ consequences       ┆ 42    │
│ conspiracy         ┆ 35    │
│ polarization       ┆ 9     │
└────────────────────┴───────┘

📈 平均置信度: 0.464


## 步骤 4: 政治立场分类

基于关键词的立场检测：
- **conservative**: 保守派（支持Kirk、谴责暴力、捍卫保守价值）
- **liberal**: 自由派（批评Kirk、反思言论、强调后果）
- **neutral**: 中立（客观报道、学术分析、悼念）

In [13]:
# 政治立场关键词
stance_keywords = {
    'conservative': [
        r'\bhero\b', r'\bpatriot\b', r'\bfreedom fighter\b', r'\bdefend\w*\b',
        r'\bMAGA\b', r'\bTrump\b', r'\bconservative movement\b',
        r'\bleft\w* violence\b', r'\bsocialist\w*\b', r'\bliberal\w* violence\b',
        r'\bmarty\w*\b', r'\bstanding up\b'
    ],
    'liberal': [
        r'\bhateful\b', r'\btoxic\b', r'\bdangerous rhetoric\b',
        r'\bextremis\w*\b', r'\bhate speech\b', r'\bconsequences\b',
        r'\bdeserve\w*\b', r'\breap what\b', r'\bfar-right\b',
        r'\bTurning Point\b.*\bnegative\b'
    ]
}

def detect_stance(text):
    """检测政治立场"""
    text_lower = text.lower()
    
    conservative_score = sum(1 for pattern in stance_keywords['conservative'] if re.search(pattern, text_lower))
    liberal_score = sum(1 for pattern in stance_keywords['liberal'] if re.search(pattern, text_lower))
    
    if conservative_score > liberal_score and conservative_score > 0:
        return 'conservative', conservative_score
    elif liberal_score > conservative_score and liberal_score > 0:
        return 'liberal', liberal_score
    else:
        return 'neutral', 0

print("🎯 开始政治立场分类...")
stance_results = [detect_stance(t) for t in texts]
stances = [r[0] for r in stance_results]
stance_scores = [r[1] for r in stance_results]

df_sample = df_sample.with_columns([
    pl.Series('political_stance', stances),
    pl.Series('stance_confidence', stance_scores)
])

print(f"✅ 政治立场分类完成")
print(f"\n📊 立场分布:")
print(df_sample.group_by('political_stance').agg(pl.len().alias('count')).sort('count', descending=True))

🎯 开始政治立场分类...
✅ 政治立场分类完成

📊 立场分布:
shape: (3, 2)
┌──────────────────┬───────┐
│ political_stance ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
│ neutral          ┆ 1843  │
│ liberal          ┆ 88    │
│ conservative     ┆ 69    │
└──────────────────┴───────┘


## 步骤 5: 时间演变分析

分析情感与叙事在5个时间窗口的演变

In [14]:
print("📈 时间演变分析")

# 情感随时间演变
emotion_evolution = df_sample.group_by('time_window').agg([
    pl.len().alias('tweet_count'),
    pl.col('emotion_sadness').mean().alias('avg_sadness'),
    pl.col('emotion_anger').mean().alias('avg_anger'),
    pl.col('emotion_fear').mean().alias('avg_fear'),
    pl.col('emotion_surprise').mean().alias('avg_surprise'),
    pl.col('emotion_joy').mean().alias('avg_joy'),
    pl.col('emotion_love').mean().alias('avg_love')
]).sort('time_window')

print("\n🎭 情感演变 (平均分数):")
print(emotion_evolution)

# 叙事随时间演变
narrative_evolution = df_sample.group_by(['time_window', 'primary_narrative']).agg(
    pl.len().alias('count')
).sort(['time_window', 'count'], descending=[False, True])

print("\n📖 叙事演变 (各时段top3叙事):")
for window in ['0-6h', '6-12h', '12-24h', '24-48h', '48-72h']:
    top_narratives = narrative_evolution.filter(pl.col('time_window') == window).head(3)
    print(f"\n  {window}:")
    for row in top_narratives.iter_rows(named=True):
        print(f"    - {row['primary_narrative']}: {row['count']} 条")

📈 时间演变分析

🎭 情感演变 (平均分数):
shape: (1, 8)
┌─────────────┬─────────────┬────────────┬───────────┬──────────┬────────────┬──────────┬──────────┐
│ time_window ┆ tweet_count ┆ avg_sadnes ┆ avg_anger ┆ avg_fear ┆ avg_surpri ┆ avg_joy  ┆ avg_love │
│ ---         ┆ ---         ┆ s          ┆ ---       ┆ ---      ┆ se         ┆ ---      ┆ ---      │
│ str         ┆ u32         ┆ ---        ┆ f64       ┆ f64      ┆ ---        ┆ f64      ┆ f64      │
│             ┆             ┆ f64        ┆           ┆          ┆ f64        ┆          ┆          │
╞═════════════╪═════════════╪════════════╪═══════════╪══════════╪════════════╪══════════╪══════════╡
│ 48-72h      ┆ 2000        ┆ 0.144821   ┆ 0.217268  ┆ 0.192494 ┆ 0.126924   ┆ 0.074406 ┆ 0.0      │
└─────────────┴─────────────┴────────────┴───────────┴──────────┴────────────┴──────────┴──────────┘

📖 叙事演变 (各时段top3叙事):

  0-6h:

  6-12h:

  12-24h:

  24-48h:

  48-72h:
    - political_violence: 950 条
    - memorial: 660 条
    - none: 236 条


## 步骤 6: 提取代表性推文

每类叙事选择2条最具代表性的推文（基于engagement和叙事得分）

In [15]:
print("📝 提取代表性推文...")

# 计算engagement分数
df_sample = df_sample.with_columns(
    (pl.col('retweetCount') + pl.col('likeCount') * 0.5 + pl.col('replyCount') * 0.3).alias('engagement_score')
)

representative_tweets = {}

for narrative in narrative_keywords.keys():
    # 筛选该叙事的推文
    narrative_tweets = df_sample.filter(
        (pl.col('primary_narrative') == narrative) &
        (pl.col(f'narrative_{narrative}') >= 2)  # 至少匹配2个关键词
    ).sort('engagement_score', descending=True).head(2)
    
    if narrative_tweets.height > 0:
        representative_tweets[narrative] = narrative_tweets.select(['text', 'engagement_score', 'primary_emotion']).to_dicts()

print(f"\n✅ 代表性推文提取完成")
print(f"\n🏆 各叙事代表性推文（前100字符）:")
for narrative, tweets in representative_tweets.items():
    print(f"\n【{narrative.upper()}】")
    for i, tweet in enumerate(tweets, 1):
        print(f"  {i}. [{tweet['primary_emotion']}] {tweet['text'][:100]}...")
        print(f"     Engagement: {tweet['engagement_score']:.0f}")

📝 提取代表性推文...

✅ 代表性推文提取完成

🏆 各叙事代表性推文（前100字符）:


## 步骤 7: 保存分析结果

In [16]:
from src import io

# 保存完整的内容分析数据
content_path = Path("../parquet/content_analysis.parquet")
io.materialize_parquet(df_sample.lazy(), content_path)
print(f"✅ 内容分析结果已保存: {content_path}")

# 保存情感演变数据
emotion_evo_path = Path("../parquet/emotion_evolution.parquet")
io.materialize_parquet(emotion_evolution.lazy(), emotion_evo_path)
print(f"✅ 情感演变数据已保存: {emotion_evo_path}")

# 保存叙事演变数据
narrative_evo_path = Path("../parquet/narrative_evolution.parquet")
io.materialize_parquet(narrative_evolution.lazy(), narrative_evo_path)
print(f"✅ 叙事演变数据已保存: {narrative_evo_path}")

# 保存代表性推文（转为DataFrame）
repr_tweets_list = []
for narrative, tweets in representative_tweets.items():
    for tweet in tweets:
        repr_tweets_list.append({
            'narrative': narrative,
            'text': tweet['text'],
            'emotion': tweet['primary_emotion'],
            'engagement': tweet['engagement_score']
        })

if repr_tweets_list:
    repr_tweets_df = pl.DataFrame(repr_tweets_list)
    repr_tweets_path = Path("../parquet/representative_tweets.parquet")
    io.materialize_parquet(repr_tweets_df.lazy(), repr_tweets_path)
    print(f"✅ 代表性推文已保存: {repr_tweets_path}")

print(f"\n📊 数据概览:")
print(f"  总分析推文数: {df_sample.height:,}")
print(f"  时间窗口数: 5")
print(f"  情感维度: 6")
print(f"  叙事框架: 6")
print(f"  政治立场: 3")

✅ 内容分析结果已保存: ../parquet/content_analysis.parquet
✅ 情感演变数据已保存: ../parquet/emotion_evolution.parquet
✅ 叙事演变数据已保存: ../parquet/narrative_evolution.parquet

📊 数据概览:
  总分析推文数: 2,000
  时间窗口数: 5
  情感维度: 6
  叙事框架: 6
  政治立场: 3


## ✅ 内容分析完成！

**生成的核心数据**:
- `content_analysis.parquet`: 完整的情感、叙事、立场分析结果
- `emotion_evolution.parquet`: 6大情感随时间的演变
- `narrative_evolution.parquet`: 6大叙事随时间的演变
- `representative_tweets.parquet`: 各叙事的代表性推文

**下一步**: 
1. 运行 `01_temporal_dynamics.ipynb` 生成小时级时间序列
2. 构建可视化Dashboard展示所有洞察